## Explaning the structure of PaletteViewModel

PaletteViewModel implements the ViewModel part of MVVM (Model-View-ViewModel) pattern in .NET MAUI

The private fields below support palette export feature

In [ ]:
private readonly IFileSaveService _fileSaveService; // Image to local storage
private readonly Export _exportService; // general export feature

The bool below manages toolbar state for marking palette as a favorite (not yet implemented)

In [ ]:
private bool _isFavoritePalette;

[ObservableCollection](https://learn.microsoft.com/en-us/dotnet/api/system.collections.objectmodel.observablecollection-1?view=net-9.0): 
+ It is a special collection that notifies the UI when items are added, removed, or refreshed, in this case the Swatch Objects which will automatically update based on their bound conditions to UI elements


In the following snippet, the constructor hold swatch data, where each item is an instance of the Swatch model. 

In [ ]:
public ObservableCollection<Swatch> Swatches { get; set; }

The constructor also sets up the RegenerateCommand to exeucte the GeneratePalette method when triggered.

In [ ]:
public ICommand RegenerateCommand { get; }
public ICommand ExportPaletteCommand { get; }
public ICommand ToggleFavoritePaletteCommand { get; }

PaletteViewModel Constructor
<break>

Here, the constructor initializes dependencies, states, and commands of:
1. Dependency Injection: File Save and Export
2. Swatch Initialization: default colors view
3. Command InitializationL: binds UI actions to logic
4. PropertyChanged Event Setup: adds a no-op event handler to prevent null reference issues during property change notification

In [ ]:
public PaletteViewModel(IFileSaveService fileSaveService, Export exportService)
{
    _fileSaveService = fileSaveService ?? throw new ArgumentNullException(nameof(fileSaveService));
    _exportService = exportService ?? throw new ArgumentNullException(nameof(exportService));

    // Initialize swatches
    Swatches = new ObservableCollection<Swatch>
    {
        new Swatch(Colors.LightSalmon),
        new Swatch(Colors.SkyBlue),
        new Swatch(Colors.MediumSeaGreen),
        new Swatch(Colors.Goldenrod),
        new Swatch(Colors.MediumOrchid)
    };

    // Initialize commands
    RegenerateCommand = new Command(GeneratePalette);
    ToggleFavoritePaletteCommand = new Command(() => IsFavoritePalette = !IsFavoritePalette);
    ExportPaletteCommand = new Command(async () => await ExportPaletteAsync());

    // Prevent null event handlers
    PropertyChanged += (sender, args) => { };
}

The GeneratePalette method creates a new Random object to generate random numbers. it loops through each swatch in collection to create a new color from RGBvalues. It does not modify locked swatches due to the logic in Swatch.IsLocked property.

In [ ]:
private void GeneratePalette()
{
    var random = new Random();

    foreach (var swatch in Swatches)
    {
        if (swatch.IsDeleted)
        {
            swatch.Color = swatch.PreviousColor;
            swatch.IsDeleted = false;
            swatch.IsActive = false;
        }

        if (!swatch.IsLocked)
        {
            swatch.Color = Color.FromRgb(
                random.Next(256),
                random.Next(256),
                random.Next(256));
        }

        swatch.OnPropertyChanged(nameof(swatch.Color));
        swatch.OnPropertyChanged(nameof(swatch.ButtonVisible));
        swatch.OnPropertyChanged(nameof(swatch.DeleteButtonVisible));
    }
}

The [Task](https://learn.microsoft.com/en-us/dotnet/csharp/asynchronous-programming/task-asynchronous-programming-model) [Asyncrhonous](https://learn.microsoft.com/en-us/dotnet/csharp/asynchronous-programming/) Method for Saving and Exporting
<break>

- Async: enables asynchronous operations without blocking UI thread, i.g. exporting a file.
- Task: represents an asynchronous operation that does return a result, rather it runs in the background and completes at some point in the program.
<break>

(code snippet is too long so I'll break it up and comment)

In [ ]:
private async Task ExportPaletteAsync()

activeColors filters for any swatch are are not marked as deleted, then forms the palette to be exported

In [ ]:
var activeColors = Swatches
    .Where(s => !s.IsDeleted)
    .Select(s => s.Color)
    .ToList();


If there are no colors, there's nothing to export

In [ ]:
if (activeColors.Count == 0)
{
    var mainPage = Application.Current?.Windows.FirstOrDefault()?.Page;
    if (mainPage != null)
    {
        await mainPage.DisplayAlert("Export", "No swatches to export.", "OK");
    }
    return;
}

Ensures application is running and page is initialized correctly before continuing with the operation.

In [ ]:
var currentPage = Application.Current?.Windows.FirstOrDefault()?.Page;
if (currentPage == null) return;

Prompts user to choose between Sharing and Saving

In [ ]:
string action = await currentPage.DisplayActionSheet(
    "Choose export method",
    "Cancel",
    null,
    "Share Palette",
    "Save to File"
);

Executes user's selection
<break>

Within the else if action, it generates an image representing the palette, construct a default name with current date and time, and saves via injected file service

In [ ]:
if (action == "Share Palette")
{
    await SharePaletteAsync(activeColors);
}
else if (action == "Save to File")
{
    var imageBytes = await Export.GeneratePaletteImageAsync(activeColors);
    var fileName = $"palette_{DateTime.Now:yyyyMMdd_HHmmss}.png";
    await _fileSaveService.SaveFileAsync(imageBytes, fileName);
}

And if the user selects "Share Palette" the method below executes

In [ ]:
private async Task SharePaletteAsync(List<Color> activeColors)
{
    var imageBytes = await Export.GeneratePaletteImageAsync(activeColors);
    var fileName = $"palette_{DateTime.Now:yyyyMMdd_HHmmss}.png";

    var tempFilePath = Path.Combine(FileSystem.CacheDirectory, fileName);
    await File.WriteAllBytesAsync(tempFilePath, imageBytes);

    await Share.RequestAsync(new ShareFileRequest
    {
        Title = "Share Palette",
        File = new ShareFile(tempFilePath)
    });
}

Interface: [INotifyPropertyChange](https://learn.microsoft.com/en-us/dotnet/api/system.runtime.compilerservices.callermembernameattribute?view=net-9.0) implementation using OnPropertyChanged method with CallerMemberName attribute:

"Implementing the INotifyPropertyChanged interface when binding data. This interface allows the property of an object to notify a bound control that the property has changed, so that the control can display the updated information."

In [ ]:
public event PropertyChangedEventHandler? PropertyChanged;

public void OnPropertyChanged([CallerMemberName] string name = null!)
{
    PropertyChanged?.Invoke(this, new PropertyChangedEventArgs(name));
}